In [1]:
import pickle
import pandas as pd

In [7]:
responses = []

# Iterate through the pickle files and extend responses
for i in range(4):
    file_path = f"data_{i}.pkl"
    with open(file_path, "rb") as f:
        responses.extend(pickle.load(f))

print(len(responses))

131187


In [94]:
# Open test.csv
df = pd.read_csv("/home/arnav/BTP/Arnav_Medha/aml/student_resource_3/dataset/test.csv")
print(len(df))

131187


In [95]:
# Add responses to the dataframe
df["response"] = responses
df.head()

,index,image_link,group_id,entity_name,response
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,Value: 6.68 \nUnit: cm
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"Value: 42cm \nUnit: /16.54"""
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,Value: 200cm \nUnit: cm
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,Value: 200cm \nUnit: cm
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,Value: 90cm \nUnit: cm


In [96]:
# Save a new csv file
path_before_preproc = "minicpm_before_postproc.csv"
df.to_csv(path_before_preproc, index=False)

- Extracting value and unit from the generated responses

In [97]:
import re
# Open CSV before preprocessing
path_before_preproc = "minicpm_before_postproc.csv"
df_preproc = pd.read_csv(path_before_preproc)

# Get the responses column
resps = df_preproc["response"]
bad_responses = []
cnt = 0
for response in resps:
    if "Value:" in response and "Unit:" in response:
        cnt += 1
    else:
        bad_responses.append(response)

print(cnt)
for response in bad_responses:
    print(response)

131173
The voltage and its unit present on the image are not clearly visible due to the resolution and angle of the photograph. The labels on the cutting discs typically include information such as the voltage, but in this case, the details are not discernible. Therefore, it is not possible to provide the value and unit of the voltage from the image.
The image does not provide any visible information regarding wattage. Therefore, it is not possible to extract a wattage value and its unit from the image.
Value: 2M/6.6ft
Value: 120V
Value: 5g
The image does not provide any visible information regarding the wattage of the USB chargers. Therefore, it is not possible to extract a wattage value and its unit from the image.
The image does not provide a clear and specific wattage value for any of the tools. Wattage is typically a measure of power, and it is not a standard specification listed on the tools shown in the image. Therefore, it is not possible to extract a wattage value from the ima

In [98]:
def extract_value_unit(input_string):
    # Regular expressions to capture content after 'Value:' and 'Unit:'
    value_pattern = r'Value:\s*(.*?)\s*(?:\n|$)'
    unit_pattern = r'Unit:\s*(.*?)\s*(?:\n|$)'

    # Search for the patterns in the input string
    value_match = re.search(value_pattern, input_string)
    unit_match = re.search(unit_pattern, input_string)

    # Extract the captured content if matches are found
    value_content = value_match.group(1) if value_match else None
    unit_content = unit_match.group(1) if unit_match else None

    if unit_content == "'" or unit_content == '"':
        unit_content = 'inches'

    return value_content, unit_content

# Count the total number of "Value" in the responses
count = 0
values = []
units = []
bad_responses = []
for response in responses:
    if "Value:" in response and "Unit:" in response:
        count += 1
        # Extract the value and unit from the response
        value, unit = extract_value_unit(response)
        values.append(value)
        units.append(unit)
    else:
        bad_responses.append(response)
        if "Value:" in response:
            value, _ = extract_value_unit(response)
            values.append(value)
            units.append("N/A")
        else:
            values.append("N/A")
            units.append("N/A")

print(len(values))
print(len(units))
# Add thes extracted values and units to the dataframe
df_preproc["value"] = values
df_preproc["unit"] = units

# Convert to strings
df_preproc["value"] = df_preproc["value"].astype(str)
df_preproc["unit"] = df_preproc["unit"].astype(str)

131187
131187


In [99]:
# Save the dataframe to a new CSV file
path_after_preproc = "_after_postproc.csv"
df_preproc.to_csv(path_after_preproc, index=False)

In [100]:
# Open the CSV file
df_after_preproc = pd.read_csv(path_after_preproc)

# Get values and units
responses = df_after_preproc["response"]
measurements = df_after_preproc["value"]
units = df_after_preproc["unit"]

# Convert all measurements and units to strings
measurements = measurements.astype(str)
units = units.astype(str)

In [101]:
# Check rows where the measurement has a '-' or 'to'
bad_rows = []
for i, measurement in enumerate(measurements):
    if "-" in measurement or "to" in measurement:
        bad_rows.append(measurement)

# Print the bad rows
print(len(bad_rows))
print(bad_rows)


589
['208-240', '53-54cm', '110-240', '208-220', '110-240', '5-10L', '200-250lbs', '110-240', '850-880', '100-240', '200-1000', '200-1000', '50-80-100', '50-80-100', '50-80-100', '110-127', '45-180', '100-240V', '1100-1150', '1-1/8', '3-4', '110-240', '4-1/2', '208-230V~', '208-240V', '208-240', '208-240', '16-28 lbs', '12-24', '12-24', '220-240', '220-240', '208-230V', '100-240', '110-240', '110-120', '36-48', '36-48W', '100-240', '110-240', '100-277', '220-240V', '220-240', '220-240', '3-4', '110-240', '1-1/8"', '1-1/8"', '14-16', '4-12', '110-120', '3,750-8,800', '208-240V', '110-220V', '100-240', '110-240', '46-62', '46-62', '220-240', '110-240', '110-240', '110-240', '110-220V', '100-240V', '3-6', '36-48', '110-120', '110-120', '110-240V', '220-240', '220-240', '220-240', '100-240', '110-120', '110-120', '50-80-100', '50-80-100', '50-80-100', '36-48', '36-48w', '110-120', '5-6', '10-7.5', '220-240', '220-240', '50-110', '50-80-110', '50-110', '1-5/32"', '4 - 35 lbs', '100-240', '1

In [102]:
def clean_values_units(values, units):
    # Regular expression to separate numeric part from non-numeric suffix
    # This pattern accommodates decimal numbers and characters following the number
    pattern = re.compile(r'^(\d+(?:\.\d+)?)\s*([a-zA-Z]+)?\s*$')
    
    for i in range(len(values)):
        match = pattern.match(values[i])
        if match:
            # Extract the numeric part and non-numeric suffix
            numeric_part, alpha_suffix = match.groups()
            values[i] = numeric_part  # Update the value with just the numeric part
            if alpha_suffix:  # If there is a non-numeric suffix, update the unit
                units[i] = alpha_suffix.strip()
        else:
            # Attempt to handle more complex cases like '2M/6.6ft' or '53-54cm'
            complex_pattern = re.compile(r'^([0-9./-]+)\s*([a-zA-Z/]+)\s*$')
            complex_match = re.search(complex_pattern, values[i])
            if complex_match:
                values[i] = complex_match.group(1)
                units[i] = complex_match.group(2)

    return values, units

In [103]:
# Iterate through the measurements and units
measurements, units = clean_values_units(measurements, units)

In [104]:
# Write the cleaned values and units back to the dataframe
df_after_preproc["value"] = measurements
df_after_preproc["unit"] = units

df_after_preproc.tail(500)

,index,image_link,group_id,entity_name,response,value,unit
130687,130788,https://m.media-amazon.com/images/I/91UcGKCG0r...,219211,wattage,Value: 150W \nUnit: W,150,W
130688,130789,https://m.media-amazon.com/images/I/91UfWWndkZ...,911827,item_volume,Value: 10 \nUnit: FL.OZ,10,FL.OZ
130689,130790,https://m.media-amazon.com/images/I/91UiHgZo5+...,998545,item_weight,Value: 15 \nUnit: LB,15,LB
130690,130791,https://m.media-amazon.com/images/I/91UpHabxoq...,530318,maximum_weight_recommendation,Value: 80 \nUnit: lb,80,lb
130691,130792,https://m.media-amazon.com/images/I/91UpHabxoq...,530318,item_weight,Value: 80 \nUnit: lb,80,lb
...,...,...,...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation,"Value: 1,500 \nUnit: LB","1,500",LB
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight,Value: N/A \nUnit: N/A,nan,nan
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation,Value: 200 \nUnit: lbs,200,lbs
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight,Value: N/A \nUnit: N/A,nan,nan


In [105]:
# Save to a new CSV file
path_to_new_csv = "minicpm_after_after_postproc.csv"
df_after_preproc.to_csv(path_to_new_csv, index=False)

In [22]:
mapping = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

### Post-Proc Part 3 (For Values and Units)
- The heart of our solution

In [14]:
path_to_new_csv = "minicpm_after_after_postproc.csv"
# Open the CSV file
df_after_preproc = pd.read_csv(path_to_new_csv)

In [15]:
# For each entity get the unique units and add to a dictionary
# Convert all units to lowercase
df_after_preproc["unit"] = df_after_preproc["unit"].str.lower()
entities = df_after_preproc["entity_name"]
units = df_after_preproc["unit"]

entity_unit_map = {}
for entity, unit in zip(entities, units):
    if entity not in entity_unit_map:
        entity_unit_map[entity] = set()
    entity_unit_map[entity].add(unit)

# Convert to list
for entity, unit_set in entity_unit_map.items():
    entity_unit_map[entity] = list(unit_set)

# add to json
import json
with open("entity_unit_map.json", "w") as f:
    json.dump(entity_unit_map, f, indent=4)

# Print entity and length of each
for entity, unit_set in entity_unit_map.items():
    print(f"{entity}: {len(unit_set)}")

height: 179
width: 129
depth: 199
item_weight: 549
maximum_weight_recommendation: 90
wattage: 184
voltage: 162
item_volume: 180


In [16]:
import re
from fractions import Fraction
import unicodedata

# Function to convert mixed fractions to decimal values
def convert_mixed_fractions(values):
    # List to hold converted values
    converted_values = []
    
    # Regular expression for Unicode fractions and mixed fractions with slashes
    unicode_fraction_pattern = re.compile(r'(\d+)([\u00BC-\u00BE\u2150-\u215E])')
    mixed_fraction_pattern = re.compile(r'^(\d+)\s+(\d+)/(\d+)$')

    for value in values:
        # Check for Unicode fractions like "16½"
        if unicode_fraction_pattern.search(value):
            value = unicode_fraction_pattern.sub(
                lambda m: str(int(m.group(1)) + unicodedata.numeric(m.group(2))),
                value
            )
            try:
                decimal_value = float(value)
                formatted_value = f"{decimal_value:.2f}"
                converted_values.append(formatted_value)
            except ValueError:
                converted_values.append(value)
            continue

        # Check for traditional mixed fractions like "2 3/4"
        mixed_match = mixed_fraction_pattern.match(value)
        if mixed_match:
            whole_number, numerator, denominator = mixed_match.groups()
            decimal_value = int(whole_number) + float(Fraction(int(numerator), int(denominator)))
            formatted_value = f"{decimal_value:.2f}"
            converted_values.append(formatted_value)
            continue
        
        # Check for pure numeric values (integers or decimals)
        numeric_check = re.fullmatch(r'^-?\d*\.?\d*$', value.strip())
        if numeric_check:
            # Directly append values that are simple integers or decimals
            converted_values.append(value.strip())
        else:
            # Remove non-numeric characters only for conversion attempt
            clean_value = re.sub(r'[^\d.-]', '', value)
            try:
                # Convert and format if different from the original
                decimal_value = float(clean_value)
                if '.' in clean_value:
                    formatted_value = f"{decimal_value:.2f}".rstrip('0').rstrip('.')
                    converted_values.append(formatted_value if formatted_value != clean_value else value)
                else:
                    converted_values.append(value)
            except ValueError:
                # If it's not a number or convertible string, keep as is
                converted_values.append(value)

    return converted_values

# Function to extract the numeric value and unit from a string with a slash separator
def extract_value_and_unit_slash(value):
    # Regular expression to match "number + unit" potentially followed by another "number + unit" with a slash
    conversion_pattern = re.compile(r'^(\d*\.?\d+)([a-zA-Z]+)(?:\/\d*\.?\d+[a-zA-Z]+)?$')
    # Regular expression for handling simple numbers or fractions (without units)
    simple_numeric_pattern = re.compile(r'^\d+[\s\d/\d]*$')

    if conversion_pattern.match(value):
        # This extracts groups like "3.2" and "cm" from "3.2cm/8in"
        numeric_part, unit_part = conversion_pattern.match(value).groups()[:2]
        return numeric_part, unit_part
    elif simple_numeric_pattern.match(value):
        # For simple numeric values and fractions, return the value with no unit
        return value, ''
    else:
        # If it doesn't match any pattern, return two empty strings
        return '', ''

# Extract the numeric value and unit from a string with spaces
def extract_value_and_unit_spaces(value):
    # Regex to match numbers followed by units, with potential spaces and optional conversion details
    # The pattern accounts for spaces and common separators like slashes or parentheses
    conversion_pattern = re.compile(r'^(\d*\.?\d+)\s*([a-zA-Z]+)(?:[.\s]*(?:\/|\().*?$)?')

    # Try to match the input string with the refined pattern
    match = conversion_pattern.match(value.strip())
    if match:
        # Extract the number and unit from the match
        numeric_part, unit_part = match.groups()
        # Remove potential trailing punctuation from unit
        unit_part = re.sub(r'[^\w]', '', unit_part)
        return numeric_part, unit_part
    else:
        # If no match is found, return two empty strings
        return '', ''


# Get the values from the dataframe
measurements_new = list(df_after_preproc["value"].values)
units = list(df_after_preproc["unit"].values)

# Convert to strings and lowercase
measurements_new = [str(val).lower() for val in measurements_new]
units = [str(val).lower() for val in units]

# For each string convert ',' to '.' and convert mixed fractions to decimal values
for i in range(len(measurements_new)):
    measurement = measurements_new[i]
    measurement = measurement.replace(",", ".")
    measurements_new[i] = measurement

# Convert mixed fractions to decimal values
measurements_new = convert_mixed_fractions(measurements_new)

# Iterate through measurements and units together
for i in range(len(measurements_new)):
    measurement = measurements_new[i]
    unit = units[i]
    if "fl" in measurement:
        measurement = measurement.split("fl")[0].strip()
        unit = "fluid ounce"
        measurements_new[i] = measurement
        units[i] = unit
    else:
        value_temp, unit_temp = extract_value_and_unit_slash(measurement)
        if value_temp == '' and unit_temp == '':
            value_2, unit_2 = extract_value_and_unit_spaces(measurement)
            if value_2 != '' and unit_2 != '':
                measurements_new[i] = value_2
                units[i] = unit_2
            else:
                continue
        elif unit_temp != '':
            measurements_new[i] = value_temp
            units[i] = unit_temp
        else:
            continue


In [17]:
# Check rows which are non-numeric (i.e. not a float)
bad_rows = []
for i, measurement in enumerate(measurements_new):
    try:
        float(measurement)
    except ValueError:
        bad_rows.append(measurement)

# Print the bad rows
print(len(bad_rows))

# Write the bad rows to a file
with open("bad_rows.txt", "w") as f:
    for row in bad_rows:
        f.write(row + "\n")

1481


In [18]:
# Iterate through the units and entities and correct the units
corrected_units = []

for unit, entity in zip(units, entities):
    # right and left strip the unit
    unit = unit.strip()

    if entity == "width" or entity == "depth" or entity == "height":
        if unit == "foot" or unit == "feet" or unit == "ft" or unit.startswith("f"):
            corrected_units.append("foot")
        elif (unit.startswith('c') or 'cm' in unit) or ('c' in unit and not 'i' in unit or 'y' in unit):
            corrected_units.append("centimetre")
        elif (unit.startswith('i') or unit.startswith("'") or unit.startswith('"') or 'in' in unit) and not (unit.startswith('c')):
            corrected_units.append("inch")
        elif unit.startswith('y'):
            corrected_units.append("yard")
        elif unit.startswith('m'):
            # if count of m is greater than 1, then it is millimetre
            if unit.count('m') > 1:
                corrected_units.append("millimetre")
            else:
                corrected_units.append("metre")
        elif 'mm' in unit or unit.count('m') > 1:
            corrected_units.append("millimetre")
        elif 'm' in unit:
            corrected_units.append("metre")
        else:
            corrected_units.append(unit)
    elif entity == "item_weight" or entity == "maximum_weight_recommendation":
        if (unit.startswith('l')) or ('lb' in unit and not 'g' in unit) or ('pound' in unit and not 'g' in unit):
            corrected_units.append("pound")
        elif 'kg' in unit or 'kilo' in unit or 'kilogram' in unit:
            corrected_units.append("kilogram")
        elif 't' in unit and not ('b' in unit or 'r' in unit or 'z' in unit or 'm' in unit or 'g' in unit):
            corrected_units.append("ton")
        elif ('oz' in unit or 'o' in unit) and not ('r' in unit or 'g' in unit):
            corrected_units.append("ounce")
        elif 'micro' in unit:
            corrected_units.append("microgram")
        elif 'mg' in unit or 'milli' in unit or 'milligram' in unit:
            corrected_units.append("milligram")
        elif 'g' in unit and not ('b' in unit or 'h' in unit):
            corrected_units.append("gram")
        else:
            corrected_units.append(unit)
    elif entity == "voltage":
        if unit.startswith('k') and not ('w' in unit or 'p' in unit):
            corrected_units.append("kilovolt")
        elif unit.startswith('m'):
            corrected_units.append("millivolt")
        elif 'v' in unit:
            corrected_units.append("volt")
        else:
            corrected_units.append("N/A")
    elif entity == "wattage":
        if unit.startswith('k') and not ('v' in unit or 'm' in unit):
            corrected_units.append("kilowatt")
        elif 'w' in unit and not ('m' in unit or '/' in unit or 'i' in unit):
            corrected_units.append("watt")
        else:
            corrected_units.append("N/A")
    elif entity == "item_volume":
        if 'oz' in unit or 'fl' in unit or 'ounce' in unit:
            corrected_units.append("fluid ounce")
        elif 'cup' in unit:
            corrected_units.append("cup")
        elif 'q' in unit:
            corrected_units.append("quart")
        elif 'p' in unit:
            corrected_units.append("pint")
        elif 'g' in unit:
            corrected_units.append("gallon")
        elif 'ml' in unit or 'll' in unit:
            corrected_units.append("millilitre")
        elif 'f' in unit:
            corrected_units.append("cubic foot")
        elif 'in' in unit:
            corrected_units.append("cubic inch")
        elif 'imperial' in unit:
            corrected_units.append("imperial gallon")
        elif 'cl' in unit:
            corrected_units.append("centilitre")
        elif 'd' in unit:
            corrected_units.append("decilitre")
        elif 'm' in unit:
            corrected_units.append("microlitre")
        elif 'l' in unit:
            corrected_units.append("litre")
        else:
            corrected_units.append(unit)        
    else:
        corrected_units.append(unit)

In [19]:
# Update the dataframe with the corrected units
df_after_preproc["unit"] = corrected_units
df_after_preproc["value"] = measurements_new

# Save to a new CSV file
# path_to_new_csv = "minicpm_final_postproc.csv" 66.2
path_to_new_csv = "minicpm_final_postproc_two.csv"

df_after_preproc.to_csv(path_to_new_csv, index=False)

In [20]:
# Open final CSV
# path_to_new_csv = "minicpm_final_postproc.csv" 66.2
path_to_new_csv = "minicpm_final_postproc_two.csv"
df_final = pd.read_csv(path_to_new_csv)

df_final.head(10)

,index,image_link,group_id,entity_name,response,value,unit
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height,Value: 6.68 \nUnit: cm,6.68,centimetre
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width,"Value: 42cm \nUnit: /16.54""",42,centimetre
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height,Value: 200cm \nUnit: cm,200,centimetre
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth,Value: 200cm \nUnit: cm,200,centimetre
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth,Value: 90cm \nUnit: cm,90,centimetre
5,5,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,height,Value: 10.50cm \nUnit: cm,10.50,centimetre
6,6,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,width,Value: 10.50 \nUnit: cm,10.50,centimetre
7,7,https://m.media-amazon.com/images/I/11lshEUmCr...,156839,height,Value: 137.16 \nUnit: cm,137.16,centimetre
8,8,https://m.media-amazon.com/images/I/21+i52HRW4...,478357,width,Value: 40 \nUnit: cm,40,centimetre
9,9,https://m.media-amazon.com/images/I/21-LmSmehZ...,478357,height,Value: 15 \nUnit: cm,15,centimetre


In [23]:
# FINAL ROWS for appending
final_indexes = []
final_prediction = []

# Iterate through the dataframe row-wise
for i, row in df_final.iterrows():
    # Get the entity name, value and unit
    entity = row["entity_name"]
    value = row["value"]
    unit = row["unit"]
    index = row["index"]

    #print(entity, value, unit)

    # Convert all three to strings and strip
    entity = str(entity).strip()
    value = str(value).strip()
    unit = str(unit).strip()

    # Check if value contains a '-' and take the higher value
    # if "-" in value:
    #     value = value.split("-")[-1]
    #     value = str(value).strip()

    # If value contains something other than digits, decimal point or a '-', overwrite it as 'N/A'
    if not value.replace(".", "", 1).isdigit():
        value = "N/A"

    # If unit is not in the entity_unit_map, overwrite it as 'N/A'
    if unit not in mapping[entity]:
        unit = "N/A"

    # Append the index and prediction to the final lists
    if (value == "N/A" or value == "nan" or value == "") or (unit == "N/A" or unit == "nan" or unit == ""):
        final_indexes.append(index)
        final_prediction.append("")
    else:
        final_indexes.append(index)
        final_prediction.append(f"{value} {unit}")

# Print length of final indexes and final prediction
print(len(final_indexes))
print(len(final_prediction))

# Make a dataframe with the final indexes and final prediction and save to a CSV file
final_df = pd.DataFrame({"index": final_indexes, "prediction": final_prediction})

final_df.to_csv("final_predictions.csv", index=False)

131187
131187
